In [1]:
%matplotlib inline

import sys
sys.path.append("..") # This allows for importing from other directories above
import numpy as np

# Load the data
X_train = np.load('../data/pendigits_train_data.npy')
y_train = np.load('../data/pendigits_train_labels.npy')
X_test = np.load('../data/pendigits_test_data.npy')
y_test = np.load('../data/pendigits_test_labels.npy')

In [3]:
k = len(set(y_train).union(set(y_test)))
n = X_train.shape[0]


W = np.ones((n, k)) * 0.5 * (1 / n)
for i in range(n):
    W[i, y_train[i]] *= 1 / (k - 1)


print(k)
print(n)

10
7494


In [7]:
np.sum(W)

4.555555555555555

In [74]:
def one_hot_labels(y, k):
    """
    y: N by 1 numpy-array corresponding to the labels.
    k: float, corresponding to number of unique classes.
    returns: Y, an N by k numpy-array with each row containing
             a 1 for the correct class and -1 otherwise.
    """
    N = y.shape[0]
    Y = np.ones((N, k)) * -1
    for i in range(N):
        Y[i, y[i]] *= -1 # Make the correct class become +1.
    return Y

In [75]:
Y = one_hot_labels(y_train, k)

In [20]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
Y

array([[-1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.],
       [-1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.],
       [-1.,  1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.],
       [-1.,  1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.],
       [-1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.],
       [ 1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.],
       [-1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.],
       [-1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.],
       [-1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.],
       [-1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.

In [22]:
Y[0,8]

1.0

In [23]:
foo = np.multiply(Y, W)

In [27]:
np.sum(foo, axis=0)

array([-0.07918335, -0.07921003, -0.07918335, -0.08081132, -0.07918335,
       -0.08078463, -0.08078463, -0.07923672, -0.08081132, -0.08081132])

In [82]:
def stump(j, b):
    """
    Parameters
    ----------
    j : int. Index of feature to use.

    b : float. Threshold for the stump.

    Returns
    -------
    a function
        Takes in a list (or generically a 1-d numpy-array)
        and outputs 1 or -1.
    """
    def phi(x):
        """
        Parameters
        ----------
        x : 1-d numpy-array or list.

        Returns
        -------
        int, \pm 1
        """
        warning_message = "Dimension of x is smaller than the index j being split on."
        assert (len(x) - 1 >= j), warning_message
        return 1 if x[j] >= b else -1
    return phi


# This is the main function that implements the base learner
def stump_base(X,Y,W):
    """
    X: N by d, numpy-like array, training data
    Y: N by k, numpy-like array, label matrix
    W: N by k, numpy-like array, weight matrix
    returns: alpha, v, phi, gamma; Make sure v is a numpy array

    """
    # Make sure phi = stump(j^*, b_{j^*})
    d = X.shape[1]
    k = Y.shape[1]
    
    # Edge of constant classifier
    gamma_vec_init = np.sum(np.multiply(Y,W), axis = 0)
    
    # Iterate across features and keep the stump that minimizes the energy Z
    best_Z = np.inf
    for j in range(d):
        s = np.sort(X[:,j]) # Get sorted column j to find best threshold to split on
        
        (v, b, gamma) = best_stump(s, Y, W, gamma_vec_init)
        alpha = 0.5 * np.log((1 + gamma) / (1 - gamma))
        phi = stump(j, b)
        Z = np.sqrt(1 - np.square(gamma)) # TODO: Not sure about this step
        
        print(b)
        print(Z)
        
        if Z < best_Z:
            v_best = v
            phi_best = phi
            alpha_best = alpha
            gamma_best = gamma
    
    return (alpha_best, v_best, phi_best, gamma_best)
    

# This is the helper function for stump_base
def best_stump(s, Y, W, gamma_vec_init):
    """
    s: N by 1, numpy-like array, sorted column of X
    Y: N by k, numpy-like array, label matrix
    W: N by k, numpy-like array, weight matrix
    gamma_vec_init: N by 1, numpy-like array, edge of constant classifer
    returns: v, b, gamma
    """
    gamma_vec_best = np.copy(gamma_vec_init)
    gamma_vec = np.copy(gamma_vec_init)
    b_best = -1 * np.inf
    
    n = Y.shape[0]
    k = Y.shape[1]
    print(n)
    print(k)
    for i in range(n-1):
        for l in range(k):
            gamma_vec[l] = gamma_vec[l] - 2.0 * W[i,l] * Y[i,l]
        if s[i] != s[i+1]:
            print("gamma_vec")
            print(np.sum(np.abs(gamma_vec)))
            print("gamma_vec_best")
            print(np.sum(np.abs(gamma_vec_best)))
            if np.sum(np.abs(gamma_vec)) > np.sum(np.abs(gamma_vec_best)):
                gamma_vec_best = np.copy(gamma_vec)
                b_best = 0.5 * (s[i] + s[i+1])
    
    v_best = np.sign(np.copy(gamma_vec_best))
    return (v_best, b_best, np.sum(np.abs(gamma_vec_best)))

In [44]:
x = np.array([[10, 22, 16],
              [37,  8,  9],
              [12, 17,  5],
              [ 4,  2, 29]])

In [45]:
x

array([[10, 22, 16],
       [37,  8,  9],
       [12, 17,  5],
       [ 4,  2, 29]])

In [47]:
y = np.array([[0, 1, 0],
              [0, 0, 1],
              [1, 0, 0],
              [1, 0, 0]])

In [48]:
y

array([[0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0]])

In [52]:
W = np.ones((4,3)) * (1 / 12)

In [70]:
(alpha_best, v_best, phi_best, gamma_best) = stump_base(x,y,W)

-inf
0.9428090415820634
-inf
0.9428090415820634
-inf
0.9428090415820634


In [68]:
phi_best(np.array([10,22,16]))

1

In [83]:
(alpha_best, v_best, phi_best, gamma_best) = stump_base(X_train,Y,W)

7494
10
gamma_vec
0.4432345876701683
gamma_vec_best
0.8000000000000743
gamma_vec
0.4355484387510321
gamma_vec_best
0.8000000000000743
gamma_vec
0.42359220709904244
gamma_vec_best
0.8000000000000743
gamma_vec
0.41569255404326355
gamma_vec_best
0.8000000000000743
gamma_vec
0.4037363223912739
gamma_vec_best
0.8000000000000743
gamma_vec
0.3928476114224976
gamma_vec_best
0.8000000000000743
gamma_vec
0.38302642113693464
gamma_vec_best
0.8000000000000743
gamma_vec
0.36893514811851824
gamma_vec_best
0.8000000000000743
gamma_vec
0.3567654123298859
gamma_vec_best
0.8000000000000743
gamma_vec
0.3420336269015415
gamma_vec_best
0.8000000000000743
gamma_vec
0.3302908993861945
gamma_vec_best
0.8000000000000743
gamma_vec
0.3166266346410637
gamma_vec_best
0.8000000000000743
gamma_vec
0.30104083266614984
gamma_vec_best
0.8000000000000743
gamma_vec
0.28502802241795144
gamma_vec_best
0.8000000000000743
gamma_vec
0.2700827328529663
gamma_vec_best
0.8000000000000743
gamma_vec
0.2538564184681253
gamma_vec_be

0.15350947424604222
gamma_vec_best
0.8000000000000743
gamma_vec
0.18147851614622867
gamma_vec_best
0.8000000000000743
gamma_vec
0.2137176407792681
gamma_vec_best
0.8000000000000743
gamma_vec
0.2427542033626678
gamma_vec_best
0.8000000000000743
gamma_vec
0.2705097411262116
gamma_vec_best
0.8000000000000743
gamma_vec
0.2980517747531128
gamma_vec_best
0.8000000000000743
gamma_vec
0.32452628769680236
gamma_vec_best
0.8000000000000743
gamma_vec
0.3490792634107097
gamma_vec_best
0.8000000000000743
gamma_vec
0.36914865225512095
gamma_vec_best
0.8000000000000743
gamma_vec
0.3892180410995322
gamma_vec_best
0.8000000000000743
gamma_vec
0.40672538030423133
gamma_vec_best
0.8000000000000743
gamma_vec
0.42679476914864256
gamma_vec_best
0.8000000000000743
gamma_vec
0.44793167867626715
gamma_vec_best
0.8000000000000743
gamma_vec
0.4654390178809663
gamma_vec_best
0.8000000000000743
gamma_vec
0.48166533226580943
gamma_vec_best
0.8000000000000743
gamma_vec
0.5013077128369353
gamma_vec_best
0.80000000000

gamma_vec
0.7572991726714998
gamma_vec_best
0.8000000000000743
gamma_vec
0.7594342140379265
gamma_vec_best
0.8000000000000743
gamma_vec
0.7615692554043533
gamma_vec_best
0.8000000000000743
gamma_vec
0.7639178009074226
gamma_vec_best
0.8000000000000743
gamma_vec
0.7662663464104921
gamma_vec_best
0.8000000000000743
-inf
0.599999999999901
7494
10
gamma_vec
0.7684013877769588
gamma_vec_best
0.8000000000000743
gamma_vec
0.7671203629571027
gamma_vec_best
0.8000000000000743
gamma_vec
0.764985321590676
gamma_vec_best
0.8000000000000743
gamma_vec
0.7615692554043932
gamma_vec_best
0.8000000000000743
gamma_vec
0.7594342140379665
gamma_vec_best
0.8000000000000743
gamma_vec
0.7575126768081824
gamma_vec_best
0.8000000000000743
gamma_vec
0.7549506271684703
gamma_vec_best
0.8000000000000743
gamma_vec
0.7515345609821875
gamma_vec_best
0.8000000000000743
gamma_vec
0.7479049906592622
gamma_vec_best
0.8000000000000743
gamma_vec
0.7444889244729794
gamma_vec_best
0.8000000000000743
gamma_vec
0.7414998665599